# 뮤지컬 데이터 수집
## 뮤지컬 50위 랭킹 데이터
# 랭킹, 뮤지컬명, 장소, 예매율 데이터 추출

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

url = "https://tickets.interpark.com/contents/ranking?genre=MUSICAL"
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')

In [74]:
#contents > article:nth-child(3) > section > div > div > div.responsive-ranking-list_rankingListWrap__GM0yK.responsive-ranking-list_topRated__axfTY > div:nth-child(2) > div.responsive-ranking-list_rankingItemInner__mMLxe > ul > div.responsive-ranking-list_rankingContentsInner__8FuZE > li
#contents > article:nth-child(3) > section > div > div > div.responsive-ranking-list_rankingListWrap__GM0yK.responsive-ranking-list_topRated__axfTY > div:nth-child(3) > div.responsive-ranking-list_rankingItemInner__mMLxe > ul > div.responsive-ranking-list_rankingContentsInner__8FuZE > li
#contents > article:nth-child(3) > section > div > div > div:nth-child(2) > div:nth-child(2) > div.responsive-ranking-list_rankingItemInner__mMLxe > ul > div.responsive-ranking-list_rankingContentsInner__8FuZE > li

In [2]:
# .responsive-ranking-list_rankingContentsInner__8FuZE > li
data = []
items = soup.select('section > div > div .responsive-ranking-list_rankingItem__PuQPJ')
len(items)

50

In [76]:
#contents > article:nth-child(3) > section > div > div > div:nth-child(2) > div:nth-child(1) > div.responsive-ranking-list_rankingItemInner__mMLxe > ul > div.responsive-ranking-list_rankingContentsInner__8FuZE > li

In [3]:
def clean_title(title):
    match = re.search(r'〈(.*?)〉', title)
    if match:
        return match.group(1).strip()
    else:
        return title.replace('뮤지컬', '').strip()

In [4]:
for idx, item in enumerate(items, start=1):
    raw_title = item.select_one(".responsive-ranking-list_goodsName__aHHGY").text.strip()
    title = clean_title(raw_title)
    place = item.select_one(".responsive-ranking-list_placeName__9HN2O").text.strip()
    rate = item.select_one(".responsive-ranking-list_bookingPercent__7ppKT").text.strip().replace("예매율", "").strip()

    data.append({
        "rank": idx,
        "title": title,
        "place": place,
        "booking_rate": rate
    })

In [5]:
data

[{'rank': 1, 'title': '위키드', 'place': '블루스퀘어 신한카드홀', 'booking_rate': '33.5%'},
 {'rank': 2, 'title': '브로드웨이 42번가', 'place': '샤롯데씨어터', 'booking_rate': '7.8%'},
 {'rank': 3, 'title': '팬텀', 'place': '세종문화회관 대극장', 'booking_rate': '7.3%'},
 {'rank': 4,
  'title': '맘마미아!',
  'place': 'LG아트센터 서울 LG SIGNATURE 홀',
  'booking_rate': '5%'},
 {'rank': 5, 'title': '멤피스', 'place': '충무아트센터 대극장', 'booking_rate': '3.8%'},
 {'rank': 6, 'title': '알라딘', 'place': '부산 드림씨어터', 'booking_rate': '3.4%'},
 {'rank': 7, 'title': '마하고니', 'place': '링크아트센터 페이코홀', 'booking_rate': '3.2%'},
 {'rank': 8, 'title': '등등곡', 'place': '예스24스테이지 1관', 'booking_rate': '2.1%'},
 {'rank': 9, 'title': '빨래', 'place': 'NOL 유니플렉스 2관', 'booking_rate': '1.6%'},
 {'rank': 10,
  'title': '스웨그에이지 외쳐,조선!',
  'place': '홍익대 대학로 아트센터 대극장',
  'booking_rate': '1.6%'},
 {'rank': 11,
  'title': '쇼맨_어느 독재자의 네 번째 대역배우',
  'place': '국립정동극장',
  'booking_rate': '1.6%'},
 {'rank': 12,
  'title': '프리다',
  'place': 'NOL 유니플렉스 1관(구 인터파크 유니플렉스)',
  'booking_

In [6]:
df = pd.DataFrame(data)
df.to_csv("musical_ranking.csv", encoding="utf-8-sig", index=False)
print("✔ CSV 저장 완료: musical_ranking.csv")

✔ CSV 저장 완료: musical_ranking.csv


In [ ]:
import pymysql

# DB 연결 설정
conn = pymysql.connect(
    host='localhost',   
    user='root',
    password='1234',
    db='web_data',
    charset='utf8mb4'
)
cursor = conn.cursor()

# 테이블 생성
create_table_sql = '''
CREATE TABLE IF NOT EXISTS musicals (
    `rank` INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255),
    place VARCHAR(255),
    booking_rate FLOAT
)
'''
cursor.execute(create_table_sql)
conn.commit()

In [ ]:
# INSERT 쿼리
insert_sql = '''
INSERT INTO musicals (`rank`, title, place, booking_rate)
VALUES (%s, %s, %s, %s)
'''

for row in data:
    cursor.execute(insert_sql, (
        row['rank'],
        row['title'],
        row['place'],
        row['booking_rate']
    ))

conn.commit()
print("✔ DB 저장 완료")

# 연결 종료
cursor.close()
conn.close()

✔ DB 저장 완료
